In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics, bwt_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize
import numpy as np
import random
import wandb
import timm
import torch
import avalanche

/home/grbagwe/miniconda3/envs/fedconlearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)


In [3]:

train_transform = Compose([
    Resize(224),
    ToTensor(),
    
])

test_transform = Compose([
    Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR100(n_experiences=10, return_task_id=False, seed=seed, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [4]:
benchmark.original_classes_in_exp

benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 'classes_order': [39,
  48,
  62,
  58,
  37,
  9,
  5,
  86,
  96,
  23,
  41,
  69,
  18,
  7,
  67,
  78,
  99,
  28,
  19,
  33,
  17,
  36,
  29,
  80,
  88,
  44,
  66,
  47,
  83,
  74,
  51,
  81,
  90,
  20,
  82,
  27,
  85,
  6,
  43,
  21,
  95,
  14,
  84,
  1,
  73,
  91,
  4,
  93,
  63

In [5]:
model = timm.models.vit_base_patch16_224_in21k(pretrained=True, num_classes=benchmark.n_classes)
model = torch.nn.DataParallel(model)

model

DataParallel(
  (module): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (dro

In [6]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
# loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar100-Vit"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#     timing_metrics(epoch=True, epoch_running=True),
#     cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
#     disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


In [7]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = EWC(
    model,
    optimizer= torch.optim.Adam(model.parameters(), lr=0.00005),
    criterion=CrossEntropyLoss(),
    ewc_lambda=1000,
#     keep_importance_data=True,
    train_mb_size=64, train_epochs=20,
    eval_mb_size=50,device='cuda',
    evaluator=eval_plugin)

In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
#     results.append(cl_strategy.eval(benchmark.test_stream))
    
